<a href="https://colab.research.google.com/github/nhbullock/Data-Science-Projects/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Trucking Company Recommender System ##
import pandas as pd
import numpy as np
from pandas import read_csv
from google.colab import drive
drive.mount ('/content/gdrive/')
companies = pd.read_csv('/content/gdrive/My Drive/Companies.csv')
user = pd.read_csv('/content/gdrive/My Drive/User_Companies.csv')

contact = ['Company','Phone number','Email','Base','Equipment','Hauler Type']
user['Equipment'] = 'Hopper'
user = user[['Company','Phone number','Email','Base','Equipment','Hauler Type']]
user.head()

companies = companies[['Company','Phone 1','Email','State','Equipment','Member Type']]
companies.columns = companies.columns.str.replace('Phone 1','Phone number')
companies.columns = companies.columns.str.replace('State','Base')
companies.columns = companies.columns.str.replace('Member Type','Hauler Type')
companies.head()

# Combine the two datasets
trucks = pd.concat([user,companies]).reset_index(drop=True)
user_data = trucks[0:224] # User data
bulkloads = trucks[225:] # BulkLoads data 
trucks.shape
trucks['Info'] = trucks[trucks.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
trucks[275:].head()

Mounted at /content/gdrive/


,Company,Phone number,Email,Base,Equipment,Hauler Type,Info
275,AC RYAN I NC,251-751-1225,ARRYAN@CENTURYTEL.NET,AL,End Dump,Carrier,"251-751-1225,ARRYAN@CENTURYTEL.NET,AL,End Dump..."
276,FASTANDRELIABLETRUCKING,469-767-3081,Fasttrucking72@yahoo.com,AL,Walking Floor,Carrier,"469-767-3081,Fasttrucking72@yahoo.com,AL,Walki..."
277,MCM TRANSPORT LLC,251-545-5320,mmoore1676@gmail.com,AL,"Pneumatic,Walking Floor",Carrier,"251-545-5320,mmoore1676@gmail.com,AL,Pneumatic..."
278,S&J'S HOTSHOTS,601-327-9294,jeffreymccardle8695@gmail.com,AL,Walking Floor,Carrier,"601-327-9294,jeffreymccardle8695@gmail.com,AL,..."
279,"SMA TRANS,LLC",256-609-8090,dewaynehghs@yahoo.com,AL,Hopper,Carrier,"256-609-8090,dewaynehghs@yahoo.com,AL,Hopper,C..."


In [ ]:
# Tfidf_Vectorizer for sparse matrix
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer()
trucks_matrix = vector.fit_transform(trucks['Info'])

# Sigmoid for tracking similiarity results sigmoid_kernel
from sklearn.metrics.pairwise import sigmoid_kernel
sk = sigmoid_kernel(trucks_matrix,trucks_matrix)

In [ ]:
## Building the model
# 1 - Get Hauler from Company Series
Company = pd.Series(trucks.index, index=trucks['Company']).drop_duplicates()
Company

Company
JBS Carriers                 0
Schwend                      1
Alexander Transport          2
GreenTop                     3
Bay & Sons                   4
                          ... 
BRADY TRUCKING INC.       7542
DOWNS TRUCKING            7543
WOODARD TRUCKING          7544
J R SANDBERG FARMS LLC    7545
DIAMOND J EXPRESS LLC     7546
Length: 7547, dtype: int64

In [ ]:
# 2 - Feed into Sigmoid to get similar scores
sk[Company['Alexander Transport']]

array([0.76159578, 0.76159562, 0.76162291, ..., 0.7615946 , 0.76159508,
       0.76159524])

In [ ]:
# 3 - Build list of sigmoid scores from truckers
def trucker(company,num, sk=sk):
  trucker_scores = pd.DataFrame(Company)
  trucker_scores['scores'] = sk[Company[company]]
  trucker_scores = trucker_scores.sort_values(by='scores',ascending=False)
  trucker_scores = trucker_scores[1:num+1]
  return trucker_scores
trucker('Alexander Transport',10)


,0,scores
Company,,
"G.R.E.L TRUCKING, LLC",1188,0.761604
AGUILAR TRUCKING,6986,0.761601
BULK COMMODITIES LLC,2459,0.761601
ENERGY RESOURCES,6153,0.761601
BOOR SERVICE TRUCKING,3021,0.761600
WILLIAMS ENTERPRISES LLC.,237,0.761600
NORTHWEST TRANSPORTATION SERVICES INC,518,0.761600
SHEENA BENNETT AND WILLIE GOODMAN DBA G&G TRUCKING,1372,0.761600
L&D Trucking,189,0.761600


In [ ]:
# 4 - Use sigmoid scores to build list of similar companies
def trucker_idx(company,num, sk=sk):
  trucker_scores = trucker(company, num)
  index = trucker_scores[0]
  return index

trucker_idx('Alexander Transport',10)

Company
G.R.E.L TRUCKING, LLC                                 1188
AGUILAR TRUCKING                                      6986
BULK COMMODITIES LLC                                  2459
ENERGY RESOURCES                                      6153
BOOR SERVICE TRUCKING                                 3021
WILLIAMS ENTERPRISES LLC.                              237
NORTHWEST TRANSPORTATION SERVICES INC                  518
SHEENA BENNETT AND WILLIE GOODMAN DBA G&G TRUCKING    1372
L&D Trucking                                           189
AMERICUS AG HAULERS INC                               1116
Name: 0, dtype: int64

In [ ]:
# 4 - Use indices to return list of trucking companies
def call_list(company,num, sk=sk):
  index = trucker_idx(company,num)
  call_list = trucks.iloc[index]
  call_list = call_list[['Company','Phone number','Email','Base','Equipment','Hauler Type']]
  return call_list

call_list('Red Horizon',10)

,Company,Phone number,Email,Base,Equipment,Hauler Type
3927,LAKE VIEW FREIGHT,320-766-3037,LENBOOGAARD81@GMAIL.COM,MN,Hopper High Side,Carrier
4001,PETERSON TRUCKING,320-766-6161,AARONPETERSON44@GMAIL.COM,MN,"Hopper,Hopper High Side,Hopper Low Side",Carrier
3685,TERRA AG,320-766-5086,derrickpearsal@gmail.com,MN,"Hopper,Hopper High Side,Hopper Low Side",Carrier Shipper
5257,M SNOBERGER FARMS INC,402-694-9647,MCSFARMSINC71@GMAIL.COM,NE,Hopper High Side,Carrier
1155,G&T WILLIAMS TRUCKING,786-521-9647,GTTRUCKING33@gmail.com,GA,Hopper,Carrier
6950,HOGGING THE LANE,903-617-9647,bobbyosborn44@yahoo.com,TX,Hopper,Carrier
3599,TEAL TRUCKING,218-766-2360,JJTEAL78@GMAIL.COM,MN,Hopper,Carrier
3655,DUSTIN BODE TRUCKING,507-766-4998,dustinbodetrucking@gmail.com,MN,Hopper,Carrier
3623,MARTY WOODFORD TRANSPORT,218-766-2073,MARTYWODFORD@GMAIL.COM,MN,"Hopper,Hopper High Side,Hopper Low Side",Carrier
3624,MCPHEE AND SONS TRUCKING,218-766-3312,joshmcphee32@hotmail.com,MN,"Hopper,Hopper Low Side",Carrier
